In [1]:
from itertools import permutations, product

import tqdm

import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from os import path

import matplotlib.pyplot as plt

from warnings import simplefilter
from collections import OrderedDict

z = False
try:
    %cd ../utils
except:
    z=True
import model_utils
import validation_utils
import query_utils

if z:
    %cd ..

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action='ignore', category=FutureWarning)

/Users/benjaminli/Documents/coding/scires/project/utils
/Users/benjaminli/Documents/coding/scires/project


In [10]:
df = pd.read_csv('data/virus_genome/human_infecting_virus', delimiter='\t', header=None)
df[['ID', 'DNA Sequence']] = df[0].str.split(expand=True)
df = df.drop(0, axis=1)
df['isZoonotic'] = 1


df2 = pd.read_csv('data/virus_genome/Other_viruses', delimiter='\t', header=None)
df2[['ID', 'DNA Sequence']] = df2[0].str.split(expand=True)
df2 = df2.drop(0, axis=1)
df2['isZoonotic'] = 0

dfother = pd.DataFrame()
it = len(df)

# iterate through each sequence
for i in range(len(df2)):
    dfother.at[it, 'DNA Sequence']=df2.at[i, 'DNA Sequence']
    dfother.at[it, 'isZoonotic']=int(0)
    dfother.at[it, 'ID']=df2.at[i, 'ID']
    it+=1


df.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)

mergedDf = pd.concat([df, dfother], axis=0)

In [11]:
print(len(mergedDf))

9430


In [13]:
asdfX = pd.DataFrame()
asdfY = pd.DataFrame()

In [14]:

def resetkmerdict(permset)->OrderedDict:
        kmerdict = OrderedDict()
        for i in permset:
            kmerdict[i]=0
        return kmerdict

def assign_kmers_to_dict(row, permset, kmer):
    kmerdict=resetkmerdict(permset)
    st = row[2] # tune for which column the sequence is in
    for j in range(len(st)-kmer+1):
        if not st[j:j+kmer] in permset: continue
        kmerdict[st[j:j+kmer]]+=1
    return kmerdict

def getTrainParams(mergedDf, kmer):
    print(mergedDf)
    s = product('acgt',repeat = kmer)
    permset = set(["".join(x) for x in list(s)])

    l = []
    
    for row in tqdm.tqdm(mergedDf.itertuples()):
        l.append(assign_kmers_to_dict(row, permset, kmer))

    finalkmerdict=pd.DataFrame(l)
    
    # shouldn't need to fill NAs
    # mergedDf.fillna(0, inplace=True)

    X = finalkmerdict
    Y = mergedDf['isZoonotic']
    X = X.apply(lambda x: (x-x.min())/(x.max()-x.min()), axis=1)
    # print(X.head())

    global asdfX
    asdfX = X.copy()
    global asdfY
    asdfY = Y.copy()


    place = pd.concat([X, Y], axis=1)
    
    # print(place)

    place.to_csv('data/info.csv', index=False)

    return train_test_split(X, Y, test_size=0.2, random_state=1)

In [7]:
for x in mergedDf['isZoonotic'].isna():
    x and print(x)

In [15]:
X_train, X_test, y_train, y_test = getTrainParams(mergedDf, 4)
zz = X_train.head()

           ID                                       DNA Sequence  isZoonotic
0       37121  cttttaacaatcatagttttataaaagggtgtaaccgaagcggttt...         1.0
1       37121  cttttaacaatcatagttttataaaagggtgtaaccgaagcggttt...         1.0
2       37121  cttttaacaatcatagttttataaaagggtgtaaccgaagcggttt...         1.0
3      129724  atactttacaattatcttgtaaaaagtagggtgtaaccgaaaaggg...         1.0
4       59303  atggcggacgtgtgacatcaccgttcgctctttctaggatcctttg...         1.0
...       ...                                                ...         ...
9425   268315  ggatccacagaactccttgtatgtacagccgcgggtacccacagca...         0.0
9426   130329  ctcatgtcattaataagaccatgcagaaaatgcaaatgaggcgaag...         0.0
9427  1239437  ccgaagtgggatggcggctgccgccgcttcggcgcttggcgcttcg...         0.0
9428   358812  gtatacgaggttagttcattctcgtatacacgattggacaaatcaa...         0.0
9429    11287  atggaaggaggaattcgtgcagcgttttcaggcctgaatgatgtta...         0.0

[9430 rows x 3 columns]


9430it [01:39, 94.81it/s] 


       cctg      gtgt      acac      ccga      ttga      accg      ctca  \
0  0.297872  0.819149  0.446809  0.074468  0.265957  0.063830  0.106383   
1  0.297872  0.819149  0.446809  0.074468  0.265957  0.063830  0.106383   
2  0.297872  0.819149  0.446809  0.074468  0.265957  0.063830  0.106383   
3  0.367347  0.857143  0.469388  0.040816  0.295918  0.081633  0.081633   
4  0.483871  0.451613  0.580645  0.494624  0.311828  0.677419  0.129032   

       tgtt      agcc      cgtt  ...      tggg      aata      tatg      gatc  \
0  0.712766  0.095745  0.085106  ...  0.372340  0.702128  0.617021  0.095745   
1  0.712766  0.095745  0.085106  ...  0.372340  0.702128  0.617021  0.095745   
2  0.712766  0.095745  0.085106  ...  0.372340  0.702128  0.617021  0.095745   
3  0.806122  0.214286  0.102041  ...  0.285714  0.581633  0.744898  0.051020   
4  0.204301  0.365591  0.129032  ...  0.204301  0.161290  0.161290  0.311828   

       ttac      ggga      ggtc      atac      atta      ccgc  
0  0